In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data

In [2]:
import pandas as pd

# file path
path = '/content/drive/MyDrive/HD/Lab/Kaggle/2-LLM_Prompt_Recovery/data/nbroad-v1.csv'
# read file
df = pd.read_csv(path)
df.head()

,id,original_text,rewrite_prompt,rewritten_text
0,LNpAovroGe,"This quilt, that my mother made, \n \n Still m...",Regency Romance: Model the text on a Regency r...,"The softest brown and brightest blue quilt, cr..."
1,nnuxwwThWi,It's the job of our agency to keep track of th...,Write like Ernest Hemingway: Focus on Hemingwa...,The agency's responsibility is to track and co...
2,aYmnFCsjKl,"The first punch gets me right in the ribs, kno...",Grimm's Fairy Tales: Adapt the text to mimic t...,"In the sweltering sun, the stench of sweat and..."
3,ufIVkreRND,Some nights I lay awake staring at the ceiling...,High Fantasy Epic: Transform the essay into a ...,In the tapestry of the ethereal realm of Eldri...
4,XwLNuYdDdE,"I can hardly read the letter, because the hand...",Fairy Tale Villain: Use the menacing and craft...,"My hand quivered as I clutched the letter, the..."


In [3]:
original_text = list(df['original_text'])
rewritten_text = list(df['rewritten_text'])
rewrite_prompt = list(df['rewrite_prompt'])

In [4]:
# test
original_text[0]
# rewritten_text[0]
# rewrite_prompt[0]

"This quilt, that my mother made, \n \n Still makes me think to this day. \n \n It's softest brown, and brightest blue, \n \n The curved stitch here, reads `` made it May''. \n \n It's hard to see, but believe me it's true, \n \n That's not just a cloth but a piece of shirt. \n \n You can see a logo here, and right there, \n \n And a signature over there, someone named `` Bert''. \n \n This is my favorite part, a piece from a stuffed bear. \n \n I think it was my mother's favorite too, \n \n She always said so at least. \n \n Something from when she was two, \n \n Given by her grandad for Thanksgiving feast. \n \n My dad added this, a little button pin, \n \n Something from his mother, for being a scout. \n \n Apparently she went to a store and fished in a bin, \n \n Until night that day, to teach him what love was about. \n \n I'm sorry you had to see this, \n \n but their funeral was delayed. \n \n \n \n\n"

## By an LLM

In [4]:
# install transformers from source
!pip -q install git+https://github.com/huggingface/transformers
!pip install --upgrade transformers > log.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import torch
import time
from transformers import pipeline

# load model and tokenizer
model = "HuggingFaceH4/zephyr-7b-alpha"
tokenizer = "HuggingFaceH4/zephyr-7b-alpha"

In [6]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device='cuda')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [7]:
# test one sentence
prompt =f"original_text:{original_text[0]}"\
        f"rewritten_text:{rewritten_text[0]}"\
        "prompt:"

res = pipe(prompt, max_new_tokens=200, pad_token_id=pipe.tokenizer.eos_token_id)

In [8]:
res

[{'generated_text': 'original_text:This quilt, that my mother made, \n \n Still makes me think to this day. \n \n It\'s softest brown, and brightest blue, \n \n The curved stitch here, reads `` made it May\'\'. \n \n It\'s hard to see, but believe me it\'s true, \n \n That\'s not just a cloth but a piece of shirt. \n \n You can see a logo here, and right there, \n \n And a signature over there, someone named `` Bert\'\'. \n \n This is my favorite part, a piece from a stuffed bear. \n \n I think it was my mother\'s favorite too, \n \n She always said so at least. \n \n Something from when she was two, \n \n Given by her grandad for Thanksgiving feast. \n \n My dad added this, a little button pin, \n \n Something from his mother, for being a scout. \n \n Apparently she went to a store and fished in a bin, \n \n Until night that day, to teach him what love was about. \n \n I\'m sorry you had to see this, \n \n but their funeral was delayed. \n \n \n \n\nrewritten_text:The softest brown an

In [9]:
res[0]['generated_text'].split("prompt:")[1].strip()

'Write a descriptive paragraph about a favorite childhood toy, focusing on sensory details and specific memories associated with it.'